In [1]:
pip install pdfplumber sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pdfplumber
import re
from sentence_transformers import SentenceTransformer, util

# Étape 1 : Extraction de texte
def extract_text_from_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# Étape 2 : Titres connus par catégorie
SECTION_KEYWORDS = {
    "experience": [
        "expérience", "expériences professionnelles", "work experience", "professional experience", "career history"
    ],
    "education": [
        "formation", "études", "éducation", "academic background", "qualifications", "diplômes"
    ],
    "skills": [
        "compétences", "skills", "technical skills", "technologies", "outils"
    ],
    "languages": [
        "langues", "languages", "spoken languages"
    ],
    "certifications": [
        "certifications", "certification", "certified"
    ],
    "summary": [
        "profil", "summary", "about me", "présentation", "professional summary"
    ],
    "contact": [
        "contact", "informations personnelles", "coordonnées", "personal information"
    ]
}

# Étape 3 : Modèle d'embedding pour matcher les titres flous
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def detect_section_title(line):
    line_clean = line.strip().lower()
    line_emb = model.encode(line_clean, convert_to_tensor=True)
    best_score = 0.7  # seuil de similarité minimum
    best_section = None

    for section, keywords in SECTION_KEYWORDS.items():
        for kw in keywords:
            kw_emb = model.encode(kw.lower(), convert_to_tensor=True)
            sim = util.pytorch_cos_sim(line_emb, kw_emb).item()
            if sim > best_score:
                best_score = sim
                best_section = section

    return best_section

# Étape 4 : Découper le texte par section
def segment_cv_text(text):
    lines = text.splitlines()
    sections = {}
    current_section = None
    buffer = []

    for line in lines:
        if not line.strip():
            continue  # ignorer les lignes vides

        possible_section = detect_section_title(line)

        if possible_section:
            if current_section:
                if current_section not in sections:
                    sections[current_section] = []
                sections[current_section].extend(buffer)
            buffer = []
            current_section = possible_section
        else:
            buffer.append(line)

    # Ajouter la dernière section
    if current_section:
        if current_section not in sections:
            sections[current_section] = []
        sections[current_section].extend(buffer)

    # Nettoyer les résultats
    for sec in sections:
        sections[sec] = "\n".join(sections[sec]).strip()

    return sections

# Exemple d'utilisation
def process_cv(path_to_pdf):
    text = extract_text_from_pdf(path_to_pdf)
    segmented_sections = segment_cv_text(text)
    return segmented_sections

# Exemple : tester avec un fichier
if __name__ == "__main__":
    pdf_path = "se.pdf"  # ← Remplace par ton fichier
    result = process_cv(pdf_path)

    # Afficher les sections détectées
    for section, content in result.items():
        print(f"\n=== {section.upper()} ===\n")
        print(content)  # Affiche les 1000 premiers caractères


2025-07-24 23:14:53.552946: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats



=== SUMMARY ===

Full-Stack Software Engineer with experience building scalable, AI-powered web applications using Java, JavaScript, React, Spring Boot, FastAPI, and
AWS. Eager to contribute in dynamic environments.

=== EXPERIENCE ===

Full Stack Software Engineer|Expersi|Paris|December 2023 - Present
•Contributed to the development and maintenance of scalable backend solutions and REST APIs using Python (FastAPI) and Java (Spring Boot), and
assisted in building front-end components with ReactJS, Selenium for data ingestion of 5 platforms and pipeline support and AWS.
•Participated in the implementation of an internal AI platform integrating LLMs and RAG pipelines, leveraging AWS services such as Bedrock,
Redshift, Lambda, API Gateway, and S3.
• Fine-tuning (4 times) and deploying custom machine learning models into production to enable search and data retrieval features.
•Fixed bugs and implemented new features while following clean code principles and established design patterns.
•

In [2]:
pdf_path = "ah.pdf"  # ← Remplace par ton fichier
result = process_cv(pdf_path)

# Afficher les sections détectées
for section, content in result.items():
    print(f"\n=== {section.upper()} ===\n")
    print(content)  # Affiche les 1000 premiers caractères


=== SUMMARY ===

16 ans d’expérience
Directeur IT et Chef de projet expérimenté, certifié Agile Scrum PSPO I, avec plus de 16 ans d’expérience dans la
gestion de projets stratégiques et la conduite de transformations digitales. Reconnu pour ma rigueur, ma capacité à
piloter des équipes pluridisciplinaires et multiculturelles, j’interviens avec efficacité sur des environnements
complexes, en garantissant un haut niveau de qualité, de performance et de conformité.
§ Estimer les charges, Allouer les ressources.
§ Suivre la réalisation, préparer les rapports de suivi, Identifier les écarts et prévoir les
risques.
§ Préparer la stratégie de livraison
§ Assurer la livraison (OPS, change management ….)
§ Assurer COMEX-COSTRAT-COPROJ ….
§ Assurer la coordination entre les équipes RH, OPS, ETL, Juridique, Sureté, Éditeurs
de solutions et fournisseurs ….
§ Assurer le passage des projets aux différentes comités de validation.
Outils : § SAP BO
§ Kibana
§ JIRA – confluence
§ MS Project
de Avril. 

In [3]:
pdf_path = "aa.pdf"  # ← Remplace par ton fichier
result = process_cv(pdf_path)

# Afficher les sections détectées
for section, content in result.items():
    print(f"\n=== {section.upper()} ===\n")
    print(content)  # Affiche les 1000 premiers caractères


=== SUMMARY ===

Ingénieur QA senior avec une expérience considérable (8ans) dans la mise en œuvre de stratégies d'amélioration continue
pour la qualité logicielle parallèlement aux meilleures pratiques d'agilité. Avec une bonne expérience en programmation,
j'ai mis en place des framework d'automatisation, des outils d'intégration et des systèmes de monitoring qui collectent
efficacement les données et garantissent une approche fluide et solide du shift-left.
Je propose également des services de formation aux outils d'automatisation des tests comme Selenium, Cypress et Katalon.
Consultant QA chez Byron Group
Du 04/2018 au 04/2019
Autant que consultant externe de Byron Group, je suis intervenu chez plusieurs clients pour des missions de mise en place
de framework de tests automatisés, notamment le client Younited Credit dans le secteur bancaire et Accor dans le secteur
d’hotelerie. L’objectif étant la validation de la plateforme UI ainsi qu’aux tests API en backend, la création de la
d